In [ ]:
#Utils:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.nonparametric.kernel_regression import KernelReg
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import random

In [3]:
#Load data
sp_comp = pd.read_csv('data/sp500_companies.csv')
sp_index = pd.read_csv('data/sp500_index.csv')
sp_stocks = pd.read_csv('data/sp500_stocks.csv')

# Data & Background

#### Data Description
This project uses three related datasets describing the S&P 500 index and its constituent companies. The S&P 500 is a market capitalization-weighted index of the 500 largest publicly listed companies trading in the United States. Market Capitalization-weighted means the share of the contributing stocks to the index is directly proportional to its relative market capitalization. The S&P 500 calculates a daily value (NAV) based on the weighted average of its components and is rebalanced quarterly. There are interquartile changes arising from new listings, delistings, and other corporate actions. Also, some stocks do not appear throughout the entire dataset as they may have experienced a valuation drawdown, listed publicly after 2010, or may have been taken private during the time period we are analyzing. The S&P 500 generally sees ~5% annual turnover and ~33% turnover per decade. Our data comes from Kaggle, and is compiled into an open source CSV format. The underlying stock prices and daily index value were collected from Yahoo Finance and the Federal Reserve Bank of St. Louis (FRED). The data was compiled primarily for research purposes as compiling historic stock price data can be a laborious and expensive task without access to proprietary databases and APIs. We are also given features of each company in the index including earnings before interest taxes and depreciation (EBITDA), market capitalization, weight to the index, geographic headquarters, among other factors.

#### Modelling Phenomena & Rational
We are modelling how stock prices change over time. The efficient market hypothesis states that all publicly available information is reflected into a stock’s price instantly, however, there is a degree of “random” chaos that cannot be incorporated prior to manifestation. Stock prices are known to be highly volatile and subject to sporadic movement due to exogenous factors beyond business quality and economic factors. While we assume that stock price movement is an independent occurrence, in reality stocks across various sectors are known to be correlated in their price movements. This is the underlying intuition behind a stock’s Beta -- or its relative volatility to the broader market’s movement. 

#### Data Exploration

In [4]:
import plotly.graph_objs as go

# testing different bandwidths
windows = [30, 90, 180, 365]
fig = go.Figure()

# Plot S&P 500
fig.add_trace(go.Scatter(
    x=sp_index["Date"],
    y=sp_index["S&P500"],
    mode='lines',
    name="S&P 500",
    line=dict(color="black", width=1)
))

# Plot moving averages
for w in windows:
    avg = sp_index["S&P500"].rolling(window=w, center=True, min_periods=w//2).mean()
    fig.add_trace(go.Scatter(
        x=sp_index["Date"],
        y=avg,
        mode='lines',
        name=f"{w}-day Moving Avg",
        line=dict(width=2)
    ))

fig.update_layout(
    title="S&P 500 Averages with bandwidths",
    xaxis_title="Date",
    yaxis_title="S&P 500",
    legend_title="Legend",
    template="plotly_white"
)
fig.show()

In [5]:
fig = px.pie(sp_comp, values='Marketcap', names='Sector', title='Marketcap by Sector')
fig.show()

# Models

### Model 1: Least Constant Squares Regression

#### Model Description

Local constant least squares regression is a non-parametric regression that looks at each point of the dependent variable only in relation to the points surrounding it. The number of points surrounding it is defined by the bandwidth parameter. The choice of bandwidth is very important to the success of this model.

A key feature of this model is that the points around it are weighted with higher weights being assigned to points closest to the target point. The function used to assign weight is called the kernel. For a local least squares regression, multiple kernels are possible. 

After the points are weighted, the weighted average of all the points within the bandwidth is assigned for prediction for the target variable. The weighted average is also called the local constant, which is also what gives this model its name.

While this model will not be difficult to code by hand due to its simplicity, but it is much easier to use statistical package such as one contained in the python libray, statsmodels.

#### Singular Example (S&P500)

In [6]:
df_companies = pd.read_csv('data/sp500_companies.csv')
df_index = pd.read_csv('data/sp500_index.csv')
df_index['Date'] = pd.to_datetime(df_index['Date'])
df_stocks = pd.read_csv('data/sp500_stocks.csv')

In [7]:
y = df_index['S&P500']
x = np.arange(len(y))
lc_model = KernelReg([y],[x],var_type='c', reg_type='lc',bw = [20])
y_pred = lc_model.fit(x)[0]
rmse = np.sqrt(np.mean((y-y_pred)**2))
mae= np.mean(np.abs(y-y_pred))
fig = px.line(
    x=df_index['Date'], 
    y=[y, y_pred], 
    labels={'x': 'Index', 'value': 'S&P500 Value', 'variable': 'Legend'},
    title='True vs Predicted S&P500 Values'
)
fig.update_traces(mode='lines')
fig.update_layout(
    legend=dict(
        title='',
        itemsizing='constant',
        traceorder='normal'
    ),
    xaxis_title='Date',
    yaxis_title='S&P500 Value'
)
fig.data[0].name = 'True S&P500'
fig.data[1].name = 'Predicted S&P500'
fig.show()

#### Bootstrapping & Evaluation

In [8]:
n_boot = 10
rmse_boot = np.zeros(n_boot)
mae_boot = np.zeros(n_boot)
n = len(y)
rng = np.random.default_rng(2)
for i in range(n_boot):
    sample = df_index.sample(n=n, replace=True, random_state=None)
    y_boot = sample['S&P500'].values
    x_boot = sample.index.values
    model_b = KernelReg([y_boot], [x_boot], var_type='c', reg_type='lc', bw=[20])
    yb_pred = model_b.fit(x)[0]
    rmse_boot[i] = np.sqrt(np.mean((y - yb_pred) ** 2))
    mae_boot[i] = np.mean(np.abs(y - yb_pred))

In [9]:
rmse_kde = ff.create_distplot(hist_data=[rmse_boot], group_labels=["RMSE"], bin_size=0.1)
rmse_kde.show()
mse_kde = ff.create_distplot(hist_data=[mae_boot], group_labels=["MAE"], bin_size=0.1)
mse_kde.show()

Comparing the simulated prices to actual prices shows that the mean prediction generally follows the real trend. Stocks with a clear underlying trend, like Apple, are captured more accurately, while volatile stocks are harder to predict. This limitation arises because the model only considers previous-day movements and does not account for longer-term seasonality or other external factors, especially random noise within the market.

### Model 2: Markov Chain Model

#### Model Description

To model the likelihood of bearish, bullish, or stable days, we split our dataset into a training set (up to 2020) and a test set (2020–2024). For each stock, we calculate the daily percent change in adjusted closing prices and classify each day into one of three states:
1. Bear: negative percent change (market goes down)
2. Bull: positive percent change (market goes up)
3. Stable: percent change within a defined threshold

#### Individual Stock Level

This classification allows us to generate state sequences for each stock across the years, forming the basis for individual stock transition matrices, which capture the probability of moving from one state to another.

In [10]:
sp_stocks = sp_stocks.sort_values(['Date','Symbol'])
#Percent Change of Adj Close from previous day to the next day for each stock
sp_stocks['pct_change'] = sp_stocks.groupby('Symbol')['Adj Close'].pct_change()
#First row will return NaN since there is no previous day to compare 
sp_stocks = sp_stocks.dropna(subset=['pct_change'])

/tmp/ipykernel_42842/423515512.py:3: FutureWarning:

The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



##### Threshold Decision

The threshold defines the range considered Stable. We experimented with values from 0.001 to 0.2. A threshold of 0.001 was small, barely reflecting stable days, while 0.2 was too large, resulting in most days being classified as stable. A threshold of 0.002 (0.2%) was chosen. This captures minor flat days as stable while keeping Bull and Bear proportions realistic.

In [11]:
threshold = 0.002

def classify_change(x):
    if x > threshold:
        return 'Bull'
    elif x < -threshold:
        return 'Bear'
    else:
        return 'Stable'
    
sp_stocks['State'] = sp_stocks['pct_change'].apply(classify_change)

sp_counts = sp_stocks['State'].value_counts(normalize=True).reset_index()

fig = px.pie(sp_counts, values='proportion', names='State', title='Proportion of Market States')
fig.show()

In [12]:
#Individual Markov chain according to each symbol/ticker
train = sp_stocks[sp_stocks['Date'] < '2020-01-01']


ticker = train['Symbol'].unique()
transition_matrices = {}

for t in ticker:
    ticker_state = train[train['Symbol'] == t].sort_values('Date')['State'].values
    transition_matrices[t] = pd.crosstab(ticker_state[:-1], ticker_state[1:], normalize='index')

##### Interpretation

From the value counts of our states, we observe that daily movements are almost evenly split between Bull and Bear days, with Stable days accounting for around 13%. Examining a single stock like CCL, we find that it is 40% likely to move from Bear to Bear, 46% from Bear to Bull, and 13% from Bear to Stable, reflecting the randomness of stock movements.

#### Sector Level Analysis

After computing individual stock transition matrices, we aggregate them to the sector level.

In [13]:
sp_sector = train.merge(sp_comp[['Symbol','Sector','Marketcap','Weight']], how='left', on='Symbol')
sector_groups = sp_sector[['Symbol','Sector']].drop_duplicates()

In [14]:
state_counts = (
    sp_sector.groupby(['Sector', 'State'])
    .size()
    .unstack(fill_value=0)
    .apply(lambda x: x / x.sum(), axis=1)
)

state_counts_reset = state_counts.reset_index().melt(id_vars='Sector', var_name='State', value_name='Proportion')

fig = px.bar(
    state_counts_reset,
    x='Sector',
    y='Proportion',
    color='State',
    title='Proportion of Bull vs Bear Days by Sector',
    barmode='stack',
    color_discrete_sequence=px.colors.sequential.Viridis
)
fig.update_layout(
    xaxis_title='Sector',
    yaxis_title='Proportion',
    legend_title='State'
)
fig.show()

We first create a dictionary of all unique sectors, such as Healthcare, Technology, and Financial Services. Each sector is assigned a final transition matrix, which is a weighted average of the transition matrices of its constituent stocks, using market capitalization as weights. This ensures that larger companies’ influence on sector behavior is accounted for.

In [15]:
# Unique sectors
sectors = sector_groups['Sector'].unique()
sector_transitions = {}

for sector in sectors:

    # Going through the tickers in each sector
    symbol_sector = sector_groups[sector_groups['Sector'] == sector]['Symbol']
    # Creating a list for matrics and weights for each sector
    matrics, weights = [], []

    # Going through the tickers in each sector
    for t in symbol_sector:

        # Going through the transition matrices for each of the tickers
        if t in transition_matrices:
            # Adding the transition matrix of the ticker to the sector list
            matrics.append(transition_matrices[t])
            # Adding the weight of the ticker to the sector list
            weight = sp_comp.loc[sp_comp['Symbol'] == t, 'Marketcap'].values[0]
            weights.append(weight)

    if matrics:

        matrics = [m.values if isinstance(m, pd.DataFrame) else m for m in matrics]

        #Weighted_sum of the matrics and weights
        weighted_sum = sum(m*w for m,w in zip(matrics, weights))
        #Averaging out the weighted sum
        avg_matrix = weighted_sum / sum(weights)
        #Adding the transition matrix of each sector to a dictionary
        states = ['Bear', 'Bull', 'Stable']
        sector_transitions[sector] = pd.DataFrame(avg_matrix, index=states, columns=states)
        # sector_transitions[sector] = avg_matrix

In [16]:
sectors_per_plot = 3
n_sectors = len(sectors)
n_cols = 3
n_rows = int(np.ceil(n_sectors / n_cols))

fig = make_subplots(
    rows=n_rows, cols=n_cols,
    subplot_titles=sectors,
    horizontal_spacing=0.08, vertical_spacing=0.12
)

states = ['Bear', 'Bull', 'Stable']

for idx, sector in enumerate(sectors):
    row = idx // n_cols + 1
    col = idx % n_cols + 1
    matrix = sector_transitions[sector].values
    heatmap = go.Heatmap(
        z=matrix,
        x=states,
        y=states,
        colorscale='tealrose',
        zmin=0, zmax=0.5,
        colorbar=dict(title='Probability'),
        showscale=(col == n_cols)  # Only show colorbar on last column
    )
    fig.add_trace(heatmap, row=row, col=col)

fig.update_layout(
    height=300 * n_rows,
    width=350 * n_cols,
    title_text="Sector Transition Matrices",
    showlegend=False
)
fig.show()

#### Interpretation

Analysis of sector-level matrices shows that most sectors display a similar trend to individual stocks. A near-even split between Bull and Bear days, with Stable days accounting for roughly 10-20%. Certain sectors, such as Utilities and Consumer Defensive, show slightly higher proportions of Stable days. Overall, the sector-level analysis aligns with the trends observed at the individual stock level.

#### Simulation and Validation

Using the test dataset (2020–2024), we simulate sector-level state sequences based on the last observed state in the training set. The simulated transition matrices are then compared to the actual transition matrices for the same period.

In [17]:
#Splitting the dataa into test and train sets
test  = sp_stocks[sp_stocks['Date'] >= '2020-01-01']

test = test.merge(sp_comp[['Symbol','Sector','Marketcap','Weight']], how='left', on='Symbol')

train = train.merge(sp_comp[['Symbol','Sector','Marketcap','Weight']], how='left', on='Symbol')

#### Simulation Phase

We take the last state from 2019 for each sector,then we simulate forward day-by-day from 2020–2024 using the training transition matrix. Each day’s next state is randomly chosen based on those pre-2020 probabilities.Finally, we compare these proportions to the real observed states.

In [18]:
def simulate_sector(start_state, transition_matrix, days):
    states = ['Bear','Bull','Stable']
    sequence = [start_state]
    for _ in range(days-1):
        probs = transition_matrix.loc[sequence[-1]].values
        next_state = random.choices(states, weights=probs, k=1)[0]
        sequence.append(next_state)
    return sequence

# Dictionary to store simulated sector states
simulated_sector_states = {}

for sector in sectors:
    # Starting state: last state from training data
    last_state = train[train['Sector']==sector].sort_values('Date')['State'].iloc[-1]
    
    # Number of days to simulate: unique dates in test for this sector
    days = test[test['Sector']==sector]['Date'].nunique()
    
    # Simulate sequence
    simulated_sector_states[sector] = simulate_sector(last_state, sector_transitions[sector], days)

# Create actual_states from the test dataset
actual_states = test[['Sector', 'State', 'Date']].copy()

# Compare simulated vs actual and collect results in a DataFrame
rows = []
for sector in sectors:
    sim_counts = pd.Series(simulated_sector_states[sector]).value_counts(normalize=True)
    actual_counts = actual_states[actual_states['Sector'] == sector]['State'].value_counts(normalize=True)
    for state in ['Bear', 'Bull', 'Stable']:
        rows.append({
            'sector': sector,
            'state': state,
            'simulated proportion': sim_counts.get(state, 0.0),
            'actual proportion': actual_counts.get(state, 0.0)
        })
sector_state_df = pd.DataFrame(rows)
sector_state_df

,sector,state,simulated proportion,actual proportion
0,Healthcare,Bear,0.412939,0.425706
1,Healthcare,Bull,0.456070,0.458928
2,Healthcare,Stable,0.130990,0.115366
3,Consumer Defensive,Bear,0.379393,0.407681
4,Consumer Defensive,Bull,0.442492,0.449414
5,Consumer Defensive,Stable,0.178115,0.142905
6,Utilities,Bear,0.373003,0.407969
7,Utilities,Bull,0.425719,0.460330
8,Utilities,Stable,0.201278,0.131700
9,Financial Services,Bear,0.394569,0.416390


#### Interpretation

The results show that the simulation is able to capture the trend of the actual proportions with slight differences, maintaining the roughly equal split between Bull and Bear days, with Stable days taking 10–20%. Stable days are slightly overestimated in the simulation.

#### Converting States to Price Movements

We convert the States to Price movement, by assigning percentage increase or decrease to each state.

Starting from the last training state, we simulate multiple price paths and compute the mean predicted price along with a 95% confidence interval.

In [19]:
# Example for a single stock/sector
states = ['Bear', 'Bull', 'Stable']
state_to_return = {'Bear': -0.015, 'Stable': 0, 'Bull': 0.015}  # example daily return

test_stock = train['Symbol'].unique()[0]  # Change to desired stock symbol


# Get the last state from training as starting point
last_state = train[train['Symbol']==test_stock].iloc[-1]['State']
transition_matrix = transition_matrices[test_stock]  # or sector_transitions['Financial Services']

n_days = len(test[test['Symbol']==test_stock])
simulations = 1000  # number of simulations for CI
simulated_prices = np.zeros((simulations, n_days))

initial_price = train[train['Symbol']==test_stock].iloc[-1]['Adj Close']

for i in range(simulations):
    price = initial_price
    state = last_state
    for day in range(n_days):
        # Choose next state based on transition probabilities
        probs = transition_matrix.loc[state].values
        state = np.random.choice(states, p=probs)
        # Convert state to price change
        price *= 1 + state_to_return[state]
        simulated_prices[i, day] = price

# Compute mean and 95% CI
mean_pred = simulated_prices.mean(axis=0)
lower = np.percentile(simulated_prices, 2.5, axis=0)
upper = np.percentile(simulated_prices, 97.5, axis=0)

# Actual test prices
actual_prices = test[test['Symbol']==test_stock]['Adj Close'].values
train_prices = train[train['Symbol']==test_stock]['Adj Close'].values

In [22]:
import plotly.graph_objs as go

fig = go.Figure()

# Plot training data
fig.add_trace(go.Scatter(
    x=list(range(len(train_prices))),
    y=train_prices,
    mode='lines',
    name='Training Data',
    line=dict(color='blue')
))

# Plot actual test data
fig.add_trace(go.Scatter(
    x=list(range(len(train_prices), len(train_prices) + len(actual_prices))),
    y=actual_prices,
    mode='lines',
    name='Actual Test Data',
    line=dict(color='green')
))

# Plot mean prediction
fig.add_trace(go.Scatter(
    x=list(range(len(train_prices), len(train_prices) + len(mean_pred))),
    y=mean_pred,
    mode='lines',
    name='Mean Prediction',
    line=dict(color='red')
))

# Plot confidence interval
fig.add_trace(go.Scatter(
    x=list(range(len(train_prices), len(train_prices) + len(mean_pred))),
    y=upper,
    mode='lines',
    name='95% Confidence Upper',
    line=dict(width=0),
    showlegend=False
))
fig.add_trace(go.Scatter(
    x=list(range(len(train_prices), len(train_prices) + len(mean_pred))),
    y=lower,
    mode='lines',
    name='95% Confidence Lower',
    fill='tonexty',
    fillcolor='rgba(255,0,0,0.3)',
    line=dict(width=0),
    showlegend=True
))

fig.update_layout(
    title=f'{test_stock} Stock Price: Markov Chain Prediction',
    xaxis_title='Days',
    yaxis_title='Price',
    legend_title='Legend',
    template='plotly_white'
)

fig.show()

#### Interpretation

Comparing the simulated prices to actual prices shows that the mean prediction generally follows the real trend. Stocks with a clear underlying trend, like Apple, are captured more accurately, while volatile stocks are harder to predict. This limitation arises because the model only considers previous-day movements and does not account for longer-term seasonality or other external factors, especially random noise within the market.

# Results & Conclusions

We observe several properties of the training data reflected in our simulated price paths. For example, simulated prices never fall below $0, and the trajectories broadly mirror the long-run trend of the training data (steady upward drift). This is expected given the train-test split and the fact that the Markov chain only models transitions between discretized return states. However, these simulations should not be interpreted as credible forecasts. The model has high uncertainty because it does not incorporate macroeconomic conditions, firm-specific information, or any structural drivers of returns. The transition probabilities are derived solely from historical day-to-day movements, so the model cannot account for exogenous shocks or shifts in market regimes. Moreover, the Markov property imposes memory-lessness: only the most recent state influences the next movement. As a result, the model cannot capture persistent dynamics like momentum, volatility clustering, or mean reversion that extend beyond a single period. The outputs therefore reflect historical transition patterns, not a realistic prediction of future prices.

The primary drawback of our model is that Markov chains are memoryless. Our model does not mimic real world exogenous factors that influence stock prices, and thereby the S&P 500. Our model assumes that the only factors that determine a future price movement are the current price and its sector, whereas in the real world, sentiment, volatility, and market events all feed into a stock’s price movement. For future work on this project, we would want to explore a parametric model that incorporates more inputs in the form of parameters that allow for memory in the model -- this would better capture a company’s qualitative and quantitative measures of performance as well as market sentiments / trends to project future price movements. For instance, transformers may allow for sentiment in the form of news headlines to be factored into future price estimations. A more robust model would allow for collection of inputs beyond just current price, which our current model does not permit. Examples of parameters include trading volume, moving average of price, company earnings transcripts, and others that would present more relevant data than just current price.
